In [1]:
from tensorflow.keras.datasets import cifar10
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.applications.vgg19 import VGG19, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import cv2
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as BE
from tensorflow.keras.losses import categorical_crossentropy
from progressbar import ProgressBar
import os
import scipy.io
import tensorflow as tf
import json
import sys
sys.path.append('..')  
from GradPri_utils.utils import *

# from tensorflow.keras import backend as BE
# from Integrated_Gradients_algorithm import *
# from GradVisualizer import *

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess=tf.compat.v1.Session(config=config) 

# tf.compat.v1.disable_eager_execution()
# tf.compat.v1.enable_eager_execution()

In [2]:
DATA_PATH = '../datasets/ImageNetVal/'
file_name = getfile_name(DATA_PATH)
file_name = np.sort(file_name)

f = open("../datasets/ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt",encoding = "utf-8")
val_ground_truth = f.read()
val_ground_truth = val_ground_truth.split('\n')
for i in range(len(val_ground_truth)-1):
    val_ground_truth[i] = int(val_ground_truth[i])

vgg19_json = json.load(open('/public/liujiawei/.keras/models/imagenet_class_index.json','r',encoding="utf-8"))

In [3]:
base_model = VGG19(weights='imagenet')

In [4]:
synsets = scipy.io.loadmat(os.path.join('../datasets', 'ILSVRC2012_devkit_t12', 'data', 'meta.mat'))['synsets']

ILSVRC2012_ID = [s[0][0][0][0] for s in synsets]

index1 = 821
WNID = [s[0][1][0] for s in synsets]
print(WNID[index1])

words = [s[0][2][0] for s in synsets]
print(words[index1])

num_train_images = [s[0][7][0][0] for s in synsets]
print(num_train_images[0])

n04263257
soup bowl
1300


In [5]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [56]:
top_set = 1 # 预测的前n个类中包含真实标签则表示预测正确
total_sample_num = 5000
img_num = range(0, total_sample_num)
ground_truth_label = []  # 记录样本的真实标签
predicted_confidence = []  # 记录样本的置信度信息
binary_flag = [] # 记录样本是否被正确分类，正确分类的标记为1，否则标记为0
binary_flag5 = []
gradients = [] # 记录样本的梯度信息
sec_gradients = [] # 记录样本的梯度信息
firProb_gradient_sum = [] # 以第一大置信度作为真实标签，计算梯度和
secProb_gradient_sum = [] # 以第二大置信度作为真实标签，计算梯度和
num_class = 1000
DATA_PATH = '../datasets/ImageNetVal/'
file_name = getfile_name(DATA_PATH)
file_name = np.sort(file_name)

pbar = ProgressBar()
for i in pbar(img_num):
    img_path = DATA_PATH + file_name[i]
    img = image.load_img(img_path, target_size=(224, 224))
    x_tmp = image.img_to_array(img)
    x_tmp = np.expand_dims(x_tmp, axis=0)
    x_tmp = preprocess_input(x_tmp)
    pre_tmp = base_model.predict(np.reshape(x_tmp, [-1,224,224,3]))
    
    predicted_confidence.append(pre_tmp)
    ground_truth_label.append(WNID[val_ground_truth[i]-1])
    binary_flag.append(
        get_acc(predict_label = decode_predictions(pre_tmp, top=top_set), 
                               ground_truth=WNID[val_ground_truth[i]-1]
               )
    )
    binary_flag5.append(
        get_acc(predict_label = decode_predictions(pre_tmp, top=5), 
                               ground_truth=WNID[val_ground_truth[i]-1]
               )
    )

100% |########################################################################|


In [70]:
shuzu = np.zeros(10)
for i in predicted_confidence:
    shuzu[int(np.max(i)*10)-1] += 1
# print(c1)
# #     i[0][np.argmax(i)]=0
# #     print(np.max(i))

In [71]:
for i in shuzu:
    print(i)

261.0
362.0
360.0
424.0
435.0
380.0
378.0
451.0
1850.0
99.0


In [98]:
shuzu = np.zeros(100)
j=0
for i in predicted_confidence:
    if binary_flag5[j] == True:
        shuzu[int(np.max(i)*100)-1] += 1
    j+=1
# print(c1)
# #     i[0][np.argmax(i)]=0
# #     print(np.max(i))

In [99]:
for i in shuzu:
    print(i)

0.0
0.0
0.0
1.0
3.0
5.0
3.0
4.0
8.0
2.0
7.0
8.0
8.0
13.0
15.0
18.0
13.0
15.0
13.0
27.0
24.0
23.0
21.0
16.0
21.0
21.0
28.0
27.0
28.0
22.0
27.0
21.0
28.0
25.0
24.0
33.0
19.0
22.0
30.0
34.0
36.0
26.0
37.0
37.0
29.0
37.0
41.0
37.0
30.0
44.0
50.0
32.0
36.0
37.0
46.0
41.0
34.0
28.0
36.0
35.0
43.0
36.0
28.0
33.0
33.0
19.0
40.0
39.0
39.0
33.0
34.0
25.0
33.0
29.0
36.0
39.0
29.0
38.0
45.0
24.0
36.0
44.0
39.0
45.0
44.0
50.0
50.0
48.0
56.0
70.0
65.0
75.0
76.0
95.0
76.0
107.0
150.0
196.0
913.0
20.0


In [147]:
a = []
for i in range(106):
    a.append(np.random.randint(8,14))
for i in a:
    print(i)

13
8
13
8
12
12
12
10
8
13
9
8
10
13
9
12
11
8
12
8
10
12
13
10
8
11
13
9
13
11
8
13
13
12
12
12
13
13
8
11
8
9
10
9
9
12
9
10
9
12
13
12
11
9
8
8
8
8
9
9
8
8
9
8
13
10
12
11
13
13
8
9
10
10
10
8
10
12
8
13
10
8
9
9
9
13
9
9
13
8
11
9
12
8
13
11
13
13
11
12
9
12
13
13
9
12


In [148]:
a = []
for i in range(256):
    a.append(np.random.randint(2,10))
for i in a:
    print(i)

3
9
6
8
2
2
7
3
8
4
6
8
4
7
7
6
4
5
2
4
6
7
2
6
5
6
7
2
8
3
2
7
5
5
7
4
7
2
3
9
3
3
4
8
9
3
2
3
9
6
8
8
7
4
3
2
9
5
5
4
5
9
6
7
2
6
5
7
6
4
3
8
4
2
9
2
8
3
6
3
8
3
3
7
5
2
7
2
5
3
8
2
4
8
5
3
3
9
9
2
3
8
6
2
9
8
8
5
3
3
9
8
5
5
7
2
7
5
3
3
2
5
5
3
6
9
8
9
9
3
8
9
6
9
7
8
5
8
2
6
2
7
5
2
2
2
8
6
6
2
4
4
5
3
9
9
3
8
6
3
5
7
5
7
2
4
9
6
8
2
7
4
6
5
8
5
3
5
6
3
3
7
6
7
3
3
6
9
6
9
5
9
4
8
6
9
8
8
8
2
5
5
4
5
2
7
5
5
3
4
3
5
8
7
5
2
4
5
8
4
2
2
3
2
2
4
2
9
8
4
6
9
5
5
6
3
7
8
6
8
7
3
8
7
7
8
5
2
9
4
4
9
7
7
2
5
